In [1]:
import pandas as pd

In [2]:
# 데이터 로드
data = pd.read_csv('./cluster_1.csv')

In [3]:
# Activity 열의 값들을 살펴보고 rating을 부여하기
data['Rating'] = data['Activity'].apply(lambda x: 5 if x == 0 else 1 + x)

In [5]:
# 데이터 저장
data.to_csv('data_rating.csv', index=False)

정규화

In [8]:
# 정규화된 rating 계산
max_rating = data['Rating'].max()
min_rating = data['Rating'].min()
data['NormRating'] = (data['Rating'] - min_rating) / (max_rating - min_rating)

In [10]:
# 파일 저장
data.to_csv('data_rating.csv', index=False)

협업필터링

In [39]:
from surprise import Dataset
from surprise import Reader

In [40]:
# rating 부여된 파일 로드
file_path = './data_rating.csv'
reader = Reader(line_format='user item rating', sep=',', rating_scale=(1, 5), skip_lines=1)
data = Dataset.load_from_file(file_path, reader=reader)

In [41]:
from surprise import KNNWithMeans
from surprise.model_selection import cross_validate
from surprise.model_selection import train_test_split

In [42]:
# 학습 데이터와 테스트 데이터로 분리
trainset, testset = train_test_split(data, test_size=0.25)

In [43]:
# KNNWithMeans 모델 생성
sim_options = {'name': 'pearson_baseline', 'user_based': True}
model = KNNWithMeans(sim_options=sim_options)

In [44]:
# 모델 학습
model.fit(trainset)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


In [45]:

# 모델 성능 평가
# cross_validate(model, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)


In [46]:
# 추천 결과 확인
user_id = str(1)  # 추천 대상 유저 ID
testset = trainset.build_testset()
user_ratings = {}
for uid, iid, r_ui in testset:
    if uid == user_id:
        user_ratings[iid] = r_ui

predictions = model.test(testset)
recommendations = []
for uid, iid, r_ui, est, _ in predictions:
    if str(uid) == user_id and iid not in user_ratings:
        recommendations.append((iid, est))
recommendations = sorted(recommendations, key=lambda x: x[1], reverse=True)[:3]
print(recommendations)

[]
